## Workbook 2: Pandas and SQL

All resources, unless otherwise specified, are created by Nikhil Chinchalkar

You've already seen some of the stuff we can do with Pandas, and we're going to expand on that with some more advanced filtering and general data cleaning skills. We'll also learn the basics of SQL (which is another language), and compare using SQL and Pandas to perform basic filtering tasks.

### A: Filtering In Pandas Vs. SQL

We saw earlier that we can do some basic filtering based on columns using some basic Pandas functions. We can also do the same stuff using SQL (and in my opinion, it's much easier). Let's start by importing `duckdb`, which is a library that allows us to run SQL (which is a different language) commands in Python.

In [3]:
import numpy as np
import pandas as pd
import duckdb

If the above gave you an error, try commenting out the line below and re-running it once the stuff below has finished.

In [ ]:
#%pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 15.0 MB/s  0:00:00 eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


We'll start by importing data from [Netflix TV Shows and Movies](https://www.kaggle.com/datasets/victorsoeiro/netflix-tv-shows-and-movies?select=titles.csv), which is available on Kaggle. This dataset shows information for all shows and movies that were available to Netflix users in July 2022.

In [4]:
titles = pd.read_csv('titles.csv')

For now, let's just do some random filtering and grouping to show the difference between Pandas and SQL. Let's say we only wanted to look at TV Shows with more than 1 season and only cared about the columns `id` and `title`. We can use the `type` variable to do so, and the filter down the columns:

In [5]:
#pandas first:
filtered_titles_PD = titles[(titles['type'] == 'SHOW') & (titles['seasons'] > 1)][['id','title']]

print(filtered_titles_PD.head())

         id                         title
5   ts22164  Monty Python's Flying Circus
35  ts20681                      Seinfeld
44  ts22082                  Knight Rider
45  ts21715              Thomas & Friends
46  ts20678             Saved by the Bell


That above Pandas syntax is a little weird and unintuitive. It doesn't exactly fly off the tongue either. Let's see the same exact task in SQL. First, I'll explain what the SQL syntax looks like. Most commands will start with a `SELECT` function, which just tells Python to take some columns `FROM` a dataframe, which is our next function. THen, there's a bunch of syntax that can follow, I'm using a `WHERE` call to specify that I want to take data *where* a condition is satisfied. That condition? where `type = 'SHOW' AND seasons > 1`. Note that the surrounding structure is important - you need the initial duckdb.sql and the triple quotes and the `df()` following to convert the task back into a dataframe.

Note that we tend to capitalize functions in SQL and leave other stuff lowercase - there's no difference in not doing that, it just makes it a bit easier to read.

In [6]:
#SQL next:
filtered_titles_SQL = duckdb.sql("""SELECT id, title FROM titles
                                 WHERE type = 'SHOW' AND seasons > 1""").df()

print(filtered_titles_SQL.head())

        id                         title
0  ts22164  Monty Python's Flying Circus
1  ts20681                      Seinfeld
2  ts22082                  Knight Rider
3  ts21715              Thomas & Friends
4  ts20678             Saved by the Bell


Both the above tasks accomplished the same thing - but one is significantly easier than the other (SQL). I'm going to continue to use SQL for most of my data cleaning tasks, including grouping, which is next:

Grouping is kind of self explanatory - you take an item in a column and put all the rows that match it together. Let's say we wanted to group by `SHOW`s and `MOVIE`s to see how many of each our original dataset has:

In [7]:
titles_count = duckdb.sql("""SELECT type, COUNT() AS Count, AVG(imdb_score) AS AverageScore 
                          FROM titles
                          GROUP BY type""").df()

print(titles_count)

    type  Count  AverageScore
0  MOVIE   3744      6.246748
1   SHOW   2106      6.977927


Though the result is pretty simple, there's a lot going on in the above call. First, we're selecting certain columns that we want to print out, including ones that we're creating based on other column data (count and average). The `AS` keyword is basically us renaming the column to something easier to interpret - you can remove the AS command to see how our output changes. Then, we specify that we want the data to be from our `titles` dataframe, and group by the `type`.

Take a second to really understand what's going on in the above call - a lot of this stuff are things that we need to do for every dataset when doing exploratory data analysis.

It's now your turn: try and do the following from the `credits` csv that you downloaded from Kaggle. Completing this step will involve some stuff from prior notebooks too.

1. Read in the CSV, saving it in a variable titled `credits`
2. Using SQL do the following (from the `credits` dataframe):
   1. Group by each `person_id` (since some actors might have the same name)
   2. Count the number of roles that each actor `person_id` is attributed to (similar to what we did above), renaming the column to `CountRoles`
   3. Use the [ORDER BY](https://www.w3schools.com/sql/sql_orderby.asp) command to sort the `CountRoles` in descending order
   4. Make sure to select the `name` and `CountRoles` columns only (you should use `FIRST(name)` in your select instead of just `name`, since there's more than one name that we're grouping by) 
   5. Save this dataframe to a variable titled `topActors`
3. Using Pandas, print the first 5 rows of `topActors`

In [27]:
#A1 your code here
#part 1 is already done for you
credits = pd.read_csv('credits.csv')
topActors = duckdb.sql("""SELECT name, 
                          Count(person_id) AS CountRoles FROM credits
                           GROUP BY person_id, name
                          ORDER BY CountRoles DESC
                          """).df()
print (topActors.head(5))

                  name  CountRoles
0  Kareena Kapoor Khan          25
1          Boman Irani          25
2       Shah Rukh Khan          23
3     Takahiro Sakurai          21
4     Amitabh Bachchan          20


Expected Output:
```
         first("name")  CountRoles
0  Kareena Kapoor Khan          25
1          Boman Irani          25
2       Shah Rukh Khan          23
3     Takahiro Sakurai          21
4         Paresh Rawal          20
```

The above section details pretty much all of the basic SQL commands, there's obviously more, but for our sake, that's a pretty solid foundation. There's one more important SQL command, though, that we need to address. Before we do that, we'll talk about NANs.

### B: Dealing With NANs

NANs, or 'Not a Number' is a type of value in Python that exists when we have a 'blank' in a dataset. Essentially, we see an NaN in a dataset when there is no number to fill in that value. We can see an NaN in practice by looking at the bottom of the `credits` dataframe, in the `character` column. Here, the character that Julian Gaviria plays is unknown/null.

In [28]:
credits.tail(5)

,person_id,id,name,character,role
77796,736339,tm1059008,Adelaida Buscato,María Paz,ACTOR
77797,399499,tm1059008,Luz Stella Luengas,Karen Bayona,ACTOR
77798,373198,tm1059008,Inés Prieto,Fanny,ACTOR
77799,378132,tm1059008,Isabel Gaona,Cacica,ACTOR
77800,1950416,tm1059008,Julian Gaviria,NaN,DIRECTOR


NaNs can be a problem for certain applications and calculations. You can't take the mean of a column with NaNs in it, and graphing NaNs can look a bit weird. 

Dealing with NaNs is a large part of the data cleaning process, and is something that depends on your application. Sometimes you might want to filter out the NaNs in one column, and leave in NaNs in another, and sometimes you might want to replace NaNs with actual values like 0s. For now, I'll just show you how to remove all NaNs in all columns, which is useful for regression applications, since most models require no NaNs as inputs.

To do so, we just use Pandas and a simple command.

In [31]:
credits_cleaned = credits.dropna()
credits_cleaned.tail(5)

,person_id,id,name,character,role
77795,368473,tm1059008,Aída Morales,Maritza,ACTOR
77796,736339,tm1059008,Adelaida Buscato,María Paz,ACTOR
77797,399499,tm1059008,Luz Stella Luengas,Karen Bayona,ACTOR
77798,373198,tm1059008,Inés Prieto,Fanny,ACTOR
77799,378132,tm1059008,Isabel Gaona,Cacica,ACTOR


Now we can see that there's no longer that NaN at the bottom of the list. Note that removing certain rows can cause the row indices to get messed up - there's no longer going to be rows at certain indices. To fix that and ensure our rows are continuous, we can just call `reset_index`, setting `drop=True` to make sure we don't add another column.

In [32]:
credits_cleaned = credits_cleaned.reset_index(drop=True)
credits_cleaned.tail(5)

,person_id,id,name,character,role
68024,368473,tm1059008,Aída Morales,Maritza,ACTOR
68025,736339,tm1059008,Adelaida Buscato,María Paz,ACTOR
68026,399499,tm1059008,Luz Stella Luengas,Karen Bayona,ACTOR
68027,373198,tm1059008,Inés Prieto,Fanny,ACTOR
68028,378132,tm1059008,Isabel Gaona,Cacica,ACTOR


We can now see that our row indices have reset, and while it's a bit difficult to show that they're continuous, just trust me that they are.

### C: Left Joins

The last SQL command we'll learn is arguably the most important one. It deals with when we want to merge two datasets together (horizontally). You might have noticed that we have two separate but similar datasets, that can be merged *on* the `id` column. That is, each actor role corresponds to a movie `id`, since each actor plays a role *in* a movie. So, we can merge the two datasets `credits` and `titles`. 

There's a bunch of ways to do that, but the easiest is to perform a left join, which means we have two datasets and a column to combine on (the column shares the same data). Then, for each column value (row) in the right dataset, we join that row in every instance of the value in the left dataset. It's easier to see what I mean in practice.

There's a way of doing this using Pandas, but like we saw above, that method is a bit unintuitive, so I'll use SQL:

In [ ]:
merged_dataset = duckdb.sql("""SELECT credits.id AS movie_id, credits.name, credits.character, titles.title ,titles.imdb_score 
                            FROM credits
                            LEFT JOIN titles ON credits.id = titles.id""").df()

merged_dataset.head()

,movie_id,name,character,title,imdb_score
0,tm84618,Robert De Niro,Travis Bickle,Taxi Driver,8.2
1,tm84618,Jodie Foster,Iris Steensma,Taxi Driver,8.2
2,tm84618,Albert Brooks,Tom,Taxi Driver,8.2
3,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,Taxi Driver,8.2
4,tm84618,Cybill Shepherd,Betsy,Taxi Driver,8.2


In the above cell, there's a lot going on. First, we're using some new notation to help Python identify which columns we actually want to take and which dataframes they actually come from. Then, we see some familiar syntax in our `FROM` statement, before we're met with another new function in `LEFT JOIN`. Here, we're just saying that we want to left join starting from `credits` (the first dataframe is the left one) `ON` the common column which corresponds to `id` in `credits` and also just happens to be `id` in `titles`. Note that they don't need to be the same name, it just so happens to be like that in this case. 

Our output here is as expected: we see that each of the rows in the right dataframe (a movie) is being matched to a corresponding row in the left dataframe (actors). That is, each actor role is being matched with the movie that the role comes from. Since there are multiple roles in each movie, it makes sense that we see Taxi Driver a bunch of times.

While this might seem a bit complicated right now, it should become second nature with some practice. Also note that there are a ton of other merging methods (right joins, inner joins, full joins, etc.) but most of the time in the data science world, you'll rarely see these other types of joins. Besides, with some manipulation you could get the equivalent of a right join with a left join, and vice versa. Essentially, yeah there's a bunch of joins, but they all can reach the same goal, which is just to have one merged dataset.

We just went over a lot of stuff, but that's pretty much all the SQL commands that I've had to use in my own data science journey, with some minor tweaks to syntax here and there. It's now time to do your own practice:

In the following code block, do the following:

1. Make a new dataframe, called `directors`, from the `credits` dataframe, only looking at directors (look at the `role` column)
2. Perform a left join on the directors dataframe (using `titles`) to match up each director to the film they directed (this should look similar to the above)
3. Use the GROUP BY attribute to group by each director's `name`
4. Use the COUNT() function to list the number of movies each director directed (rename this to `count`)
5. Use the AVG() function to find the average `imdb_score` of each of the movies (rename this to `score`)
6. Make a new dataframe from `directors` titled `top_directors` where you filter the `directors` who directed at least 5 movies
7. Sort `top_directors` by `imdb_score`, in descending order
8. Print the first 5 rows of `top_directors`

In [72]:
#C1 your code here
directors = duckdb.sql("""  
                        SELECT credits.name, COUNT(*) AS count, AVG(titles.imdb_score) as score 
                        FROM credits 
                        LEFT JOIN titles ON credits.id = titles.id
                        WHERE credits.role = 'DIRECTOR'
                        GROUP BY credits.name
                       """).df()

top_directors = duckdb.sql(""" SELECT * FROM directors 
                           WHERE count >5
                           ORDER BY score DESC""").df()

top_directors.head()

,name,count,score
0,Martin Scorsese,6,8.160000
1,Mani Ratnam,6,7.733333
2,Song Hyun-wook,6,7.375000
3,Anurag Kashyap,8,7.271429
4,Shannon Hartman,7,7.257143


Expected Output:

```
              name  count     score
0  Martin Scorsese      6  8.160000
1      Mani Ratnam      6  7.733333
2   Song Hyun-wook      6  7.375000
3   Anurag Kashyap      8  7.271429
4  Shannon Hartman      7  7.257143
```

This last assignment is pretty difficult and requires a solid foundation in SQL. If you completed it, you should feel confident about your ability to solve future problems you encounter in SQL. If you completed it and still want a challenge, try to do it in a single SQL statement (hint: you should look into the [HAVING](https://www.w3schools.com/sql/sql_having.asp) function).

Now, try and make a estimation for a frequency distribution of scores across our `titles` dataset. To do this, we'll need to do the following:

1. Select the `CEIL()` of the `imdb_score`s from the `titles` dataframe (rename this to `score`)
2. Select the `COUNT()` of the number of movies that fall under each score (rename this to `freq`)
3. `GROUP BY` each score
4. Sort the dataframe in descending order by the `score` attribute
5. Remove NaNs in the score column using the `dropna()` function (using Pandas)
6. Display the full dataframe

In [37]:
#C2 your code here

Expected Output:

```
   score  freq
0   10.0    11
1    9.0   390
2    8.0  1490
3    7.0  1835
4    6.0  1094
5    5.0   376
6    4.0   125
7    3.0    39
8    2.0     8
```

After the next lesson, you'll easily be able to plot this data in a nice histogram plot.